### Part 1: Basic Imports

In [112]:
import dask.dataframe as ddf
import dask.multiprocessing
import pandas as pd
import requests
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from time import sleep
pd.set_option('display.max_columns', None)

###  Part 2: Read the data

In [2]:
stopbyroute = pd.read_csv('dump/StopDataByRoutes.csv', header=None)

In [3]:
stopbyroute.columns = ['Route', 'Order', 'Direction', 'Stopno', 'Address', 'SeqNumber', 'lat', 'long', 'seqNumberExt']

In [7]:
stopbyroute.head()

,Route,Order,Direction,Stopno,Address,SeqNumber,lat,long,seqNumberExt
0,'1',21,I,10,Parnell Square,220,53.353387,-6.265384,220
1,'1',22,I,12,Dorset St,230,53.356789,-6.264623,230
2,'1',23,I,14,Dorset St,240,53.358537,-6.262724,240
3,'1',24,I,15,Dorset St,250,53.360251,-6.260973,250
4,'1',25,I,17,Drumcondra Rd,260,53.363067,-6.258398,260


In [8]:
stopbyroute.shape

(10956, 9)

###  Part 3: Create the last stop column and find the distance between present and last stop

The main idea is to first create a column for the last stop by looking up the last order number with same direction and route for every observation.
After that, we'll self join the table to get the lat and long for the last stop.

Then we'll query the OSRM to get the road distance between these two points.
The reason we chose to get real data and not use the straight line formula is because the formula gives a very naive result and not that accurate to mimic the real-time scenario to get delay per km.

In [9]:
# get the details (name and coordinates) of last stop for every observation of step by route
def get_last(x):
    try:
        return stopbyroute[(stopbyroute.Direction == x.Direction) & (stopbyroute.Route == x.Route) & (stopbyroute.Order == x.Order-1)]['Stopno'].values[0]
    except:
        return None

In [10]:
stopbyroute['lastStop'] = stopbyroute.apply(lambda x: get_last(x), axis=1)

In [11]:
stopbyroute.shape

(10956, 10)

In [15]:
stopbyroute[(stopbyroute.Route == "'1'") & (stopbyroute.Direction == 'O')].sort_values('Order')

,Route,Order,Direction,Stopno,Address,SeqNumber,lat,long,seqNumberExt,lastStop
30,'1',0,O,226,Shanard Road,10,53.391136,-6.262192,10,NaN
32,'1',1,O,228,Shanliss Rd,20,53.391850,-6.259797,20,226.0
33,'1',2,O,229,Shanliss Rd,30,53.391400,-6.256516,30,228.0
31,'1',3,O,227,Shanliss Rd,40,53.391143,-6.251322,40,229.0
34,'1',4,O,230,Shanliss Rd,50,53.389886,-6.249051,50,227.0
35,'1',5,O,231,Swords Road,60,53.389236,-6.245927,60,230.0
77,'1',6,O,1641,Swords Road,70,53.386524,-6.242370,70,231.0
78,'1',7,O,1642,Swords Road,80,53.384000,-6.243600,80,1641.0
21,'1',8,O,213,Swords Road,90,53.381402,-6.244888,90,1642.0
22,'1',9,O,214,Swords Road,100,53.379309,-6.246649,100,213.0


In [65]:
stopbyroute_test = stopbyroute.merge(stopbyroute[['Route', 'Stopno', 'Direction', 'lat', 'long']], 
                                     left_on=['lastStop', 'Direction', 'Route'], 
                                     right_on=['Stopno', 'Direction', 'Route'],
                                     how='left',
                                     suffixes=['', '_last'])

In [66]:
stopbyroute_test.shape

(10956, 13)

In [67]:
assert(stopbyroute_test.shape[0] == stopbyroute.shape[0])

In [68]:
stopbyroute_test.head()

,Route,Order,Direction,Stopno,Address,SeqNumber,lat,long,seqNumberExt,lastStop,Stopno_last,lat_last,long_last
0,'1',21,I,10,Parnell Square,220,53.353387,-6.265384,220,278.0,278.0,53.351628,-6.261237
1,'1',22,I,12,Dorset St,230,53.356789,-6.264623,230,10.0,10.0,53.353387,-6.265384
2,'1',23,I,14,Dorset St,240,53.358537,-6.262724,240,12.0,12.0,53.356789,-6.264623
3,'1',24,I,15,Dorset St,250,53.360251,-6.260973,250,14.0,14.0,53.358537,-6.262724
4,'1',25,I,17,Drumcondra Rd,260,53.363067,-6.258398,260,15.0,15.0,53.360251,-6.260973


### Part 4: Get distance between the present and the last stop using OSRM


#### Get the distance using OSRM (need to put a delay to stop getting blocked)

In [140]:
dist = []
error_idx = []
for i in tqdm(range(stopbyroute_test.shape[0])):
    try:
        source_coordinates = str(stopbyroute_test.loc[i,'long'])+','+str(stopbyroute_test.loc[i,'lat'])+(';') 
        dest_coordinates = str(stopbyroute_test.loc[i,'long_last'])+','+str(stopbyroute_test.loc[i,'lat_last'])

        url =  'http://router.project-osrm.org/route/v1/driving/'+source_coordinates+dest_coordinates

        payload = {"steps":"true","geometries":"geojson"}
        response = requests.get(url,params=payload)
        data = response.json()

        dist.append(data['routes'][0]['distance']/1000)
    except:
        error_idx.append(i)
        dist.append(0)
    sleep(0.5)

Once we get the data, the best way is to create a backup.

In [142]:
stopbyroute_test['dist'] = dist

In [144]:
stopbyroute_test.to_csv('stopbyroute_updated.csv', index=False)

In [159]:
stopbyroute_test[(stopbyroute_test.dist == 0) & (~stopbyroute_test.lat_last.isna())].shape

(40, 14)

#### Use the idx to retry on the error list
The main idea on the previous step was to keep track of the index for which a distance could not be retrieved. This can happen due to many reasons. But we need to retry until the number of faulty indexes remain same. This can be done using a loop, but since the runtime is huge, we cannot afford to take that risk.

In [154]:
dist1 = dist

In [156]:
error_idx1 = []
for i in tqdm(error_idx):
    try:
        source_coordinates = str(stopbyroute_test.loc[i,'long'])+','+str(stopbyroute_test.loc[i,'lat'])+(';') 
        dest_coordinates = str(stopbyroute_test.loc[i,'long_last'])+','+str(stopbyroute_test.loc[i,'lat_last'])

        url =  'http://router.project-osrm.org/route/v1/driving/'+source_coordinates+dest_coordinates

        payload = {"steps":"true","geometries":"geojson"}
        response = requests.get(url,params=payload)
        data = response.json()

        dist1[i] = data['routes'][0]['distance']/1000
    except:
        error_idx1.append(i)
        dist1[i] = 0
    sleep(0.5)

In [160]:
len(error_idx) - len(error_idx1)

24

In [161]:
stopbyroute_test['dist1'] = dist1

In [163]:
stopbyroute_test[(stopbyroute_test.dist1 == 0) & (~stopbyroute_test.lat_last.isna())].shape

(16, 15)

#### Perform another manual to retry on the error list

In [164]:
dist2 = dist1

In [165]:
error_idx2 = []
for i in tqdm(error_idx1):
    try:
        source_coordinates = str(stopbyroute_test.loc[i,'long'])+','+str(stopbyroute_test.loc[i,'lat'])+(';') 
        dest_coordinates = str(stopbyroute_test.loc[i,'long_last'])+','+str(stopbyroute_test.loc[i,'lat_last'])

        url =  'http://router.project-osrm.org/route/v1/driving/'+source_coordinates+dest_coordinates

        payload = {"steps":"true","geometries":"geojson"}
        response = requests.get(url,params=payload)
        data = response.json()

        dist2[i] = data['routes'][0]['distance']/1000
    except:
        error_idx2.append(i)
        dist2[i] = 0
    sleep(0.5)

In [166]:
len(error_idx1) - len(error_idx2)

0

Since the difference between the last two error lists is 0, it means even after running n number of retries, things won't change. So, giving up and proceeding further is by getting the rows for which no data was fetched.

In [168]:
stopbyroute_test[(stopbyroute_test.dist1 == 0) & (~stopbyroute_test.lat_last.isna())]

,Route,Order,Direction,Stopno,Address,SeqNumber,lat,long,seqNumberExt,lastStop,Stopno_last,lat_last,long_last,dist,dist1
8910,'69',52,O,4557,Rathcoole,540,53.281308,-6.469668,540,3440.0,3440.0,53.281816,-6.465497,0.0,0.0
8913,'69',32,O,4665,Convent Road,330,53.317234,-6.395502,330,2161.0,2161.0,53.318366,-6.394926,0.0,0.0
8914,'69',33,O,4666,Fonthill Rd,340,53.316819,-6.398038,340,4665.0,4665.0,53.317234,-6.395502,0.0,0.0
8915,'69',34,O,4667,Fonthill Rd,350,53.317476,-6.401319,350,4666.0,4666.0,53.316819,-6.398038,0.0,0.0
8916,'69',27,I,4672,Fonthill Rd,280,53.317547,-6.401094,280,2170.0,2170.0,53.317359,-6.403361,0.0,0.0
8917,'69',29,I,4673,Convent Road,300,53.318097,-6.395162,300,3465.0,3465.0,53.316761,-6.395815,0.0,0.0
8918,'69',1,O,4720,Aston Quay,20,53.346615,-6.260275,20,4495.0,4495.0,53.346183,-6.256966,0.0,0.0
8919,'69',15,I,5022,Kingswood Ave,160,53.293379,-6.422841,160,3438.0,3438.0,53.291424,-6.428024,0.0,0.0
8920,'69',17,I,5119,Kingswood Road,180,53.297898,-6.420124,180,6002.0,6002.0,53.295088,-6.418499,0.0,0.0
8921,'69',18,I,5120,Kingswood Road,190,53.300760,-6.418611,190,5119.0,5119.0,53.297898,-6.420124,0.0,0.0


#### Manually fill the 16 rows

Since only 16 rows are there, we can fill them manually. But taking a deep insight into the co-ordinates from OSRM, we see these rows indicate that it is not a feasible route. So, the 'dist2' column in the best suited one and we are ready with it for analysis.

In [172]:
# doing a sanity check. dist2 is completely fine and in sync with dist1
# after looking a the resukts, we see that the route '77A' is filled prefectly. Route '69' is still having the issues as seen above. So, we think it will be best if we drop this route form the analysis.
stopbyroute_test[stopbyroute_test.dist != stopbyroute_test.dist1]

,Route,Order,Direction,Stopno,Address,SeqNumber,lat,long,seqNumberExt,lastStop,Stopno_last,lat_last,long_last,dist,dist1
8893,'69',48,O,3453,Mill Road,500,53.282951,-6.452800,500,3434.0,3434.0,53.280311,-6.444680,0.0,0.7149
8894,'69',10,I,3454,Mill Road,110,53.281645,-6.447669,110,7432.0,7432.0,53.281891,-6.450144,0.0,0.1681
8895,'69',11,I,3455,Saggart Village,120,53.280342,-6.444804,120,3454.0,3454.0,53.281645,-6.447669,0.0,0.3024
8896,'69',12,I,3456,Garters Lane,130,53.283513,-6.440945,130,3455.0,3455.0,53.280342,-6.444804,0.0,0.4565
8897,'69',13,I,3457,Garters Lane,140,53.290012,-6.438245,140,3456.0,3456.0,53.283513,-6.440945,0.0,0.7544
8898,'69',21,I,3461,Boot Road,220,53.309639,-6.404421,220,5122.0,5122.0,53.306974,-6.407339,0.0,0.3599
8899,'69',28,I,3465,Convent Road,290,53.316761,-6.395815,290,4672.0,4672.0,53.317547,-6.401094,0.0,0.4503
8900,'69',25,O,4379,Naas Road,260,53.316296,-6.373706,260,1960.0,1960.0,53.320165,-6.361967,0.0,2.9333
8901,'69',41,I,4406,Naas Road,430,53.328041,-6.338472,430,1981.0,1981.0,53.326467,-6.344890,0.0,1.2996
8902,'69',52,I,4408,Conyngham Road,540,53.348327,-6.301091,540,1472.0,1472.0,53.348427,-6.305953,0.0,0.3227


In [173]:
stopbyroute_test.to_csv('stopbyroute_updated1.csv', index=False)

#### Removing route 69 from the observations

We have found out from the last to last observation that most of the stops of route 69 is faulty. So, the decision we take here is to completely remove the route from our analysis.

In [190]:
stopbyroute_test1 = stopbyroute_test.copy()

In [192]:
stopbyroute_test1 = stopbyroute_test1[stopbyroute_test1.Route!="'69'"]

In [194]:
stopbyroute_test1.shape

(10839, 15)

In [198]:
stopbyroute_test1.drop('dist', axis=1, inplace=True)

In [200]:
stopbyroute_test1.to_csv('stopbyroute_updated_final.csv', index=False)